In [3]:
from common_functions import ret_metabase,  upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, initialize_env
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import pytz

In [20]:
df = pd.read_excel('telda_CS.xlsx')
df = df[df['Source'] == 'Chat']
df['Name'] = df['Subject'].str.extract(r'Conversation with (.+)')
df = df[['Ticket ID','Description', 'Name', 'Agent' ]]
import pandas as pd
import re

# Example DataFrame

def clean_description(row):
    name = re.escape(row['Name'])
    text = row['Description']

    # Step 1: remove client segments
    cleaned = re.sub(fr'{name}[^,]*,', '', text)

    # Step 2: handle "this is {first_name}"
    if pd.notna(row['Agent']):
        first_name = row['Agent'].split()[0]  # extract first name
        pattern = fr'.*?(this\s+is\s+{re.escape(first_name)}\b)'
        match = re.search(pattern, cleaned, flags=re.IGNORECASE)
        if match:
            # Remove everything before "this is first_name"
            cleaned = cleaned[match.end():].strip()
    
    return cleaned

# Apply
df['Cleaned_Description'] = df.apply(clean_description, axis=1)

In [28]:
data = df.head(100)
data.to_excel("first_100.xlsx", index=False)

In [29]:
no_bot_data = data[~data['Description'].str.contains('Telda Bot', na=False)]
no_bot_data

,Ticket ID,Description,Name,Agent,Cleaned_Description
12,1281386,"xx_eslamshreif 01145963166 10:58 PM, 31st Aug Telda شكراّ لتواصلك مع تيلدا, نأسف جدًا لأننا نواجه ارتفاعًا في حجم المحادثات في الوقت الحالي، ونحن نقدر سعة صدرك وتفهمك. سيتم تحويلك لممثل خدمة العملاء بأقرب وقت. متوسط وقت الأنتظار هو : 4 دقائق. 10:58 PM, 31st Aug xx_eslamshreif 8057 10:58 PM, 31st Aug xx_eslamshreif المبلغ 6414 مش متذكر اووي 10:59 PM, 31st Aug xx_eslamshreif الهاتف ايفون 8بلس 10:59 PM, 31st Aug Telda نعتذر عن التأخير في التواصل مع خدمه العملاء. سيتم تعيين ممثل للخدمه لك قريبًا. 11:00 PM, 31st Aug Telda Hello Xx_eslamshreif, Thanks for reaching out to Telda! This is Sarah, I hope you are having a great day! أهلاً Xx_eslamshreif, شكراً لتواصلك مع تيلدا. أنا Sarah, أتمنى أن يكون يومك سعيد. 11:00 PM, 31st Aug xx_eslamshreif انا ارسلت كل البيانات المطلوبه 11:00 PM, 31st Aug Sarah بكل تاكيد هساعدك 11:02 PM, 31st Aug Sarah استاذنك في الانتظار 3 دقايق للمراجعه 11:02 PM, 31st Aug xx_eslamshreif في جديد 11:05 PM, 31st Aug Sarah شكرا لانتظارك وبعتذرلك لو في اي تاخير 11:06 PM, 31st Aug xx_eslamshreif ولايهمك 11:06 PM, 31st Aug Sarah دلوقتي فتحت الحساب هتتبع الخطوات التاليه للدخول مرة تانية: 1- بتكتب رقم موبايلك 2- بتختار Forget 3- بتكتب ال Act code هتلاقيه موجود ف ضهر كارت تيلدا 4- بتكتب رقمك القومي 5- بتكتب الباسورد الجديد 11:07 PM, 31st Aug Sarah اي استفسار اخر ؟ 11:08 PM, 31st Aug Sarah شكرا لتواصلك مع تيلدا 11:09 PM, 31st Aug Take to Team Inbox",xx_eslamshreif,Sarah Abdelalim,", I hope you are having a great day! أهلاً Xx_eslamshreif, شكراً لتواصلك مع تيلدا. أنا Sarah, أتمنى أن يكون يومك سعيد. 11:00 PM, 31st Aug 31st Aug Sarah بكل تاكيد هساعدك 11:02 PM, 31st Aug Sarah استاذنك في الانتظار 3 دقايق للمراجعه 11:02 PM, 31st Aug 31st Aug Sarah شكرا لانتظارك وبعتذرلك لو في اي تاخير 11:06 PM, 31st Aug 31st Aug Sarah دلوقتي فتحت الحساب هتتبع الخطوات التاليه للدخول مرة تانية: 1- بتكتب رقم موبايلك 2- بتختار Forget 3- بتكتب ال Act code هتلاقيه موجود ف ضهر كارت تيلدا 4- بتكتب رقمك القومي 5- بتكتب الباسورد الجديد 11:07 PM, 31st Aug Sarah اي استفسار اخر ؟ 11:08 PM, 31st Aug Sarah شكرا لتواصلك مع تيلدا 11:09 PM, 31st Aug Take to Team Inbox"
16,1281390,"julie_ossama Hi i ordered the card from the app but i don’t understand the sign up the document process what should i do? 11:09 PM, 31st Aug Telda Hello Julie_ossama, Thanks for reaching out to Telda! This is Mariem, I hope you are having a great day! أهلاً Julie_ossama, شكراً لتواصلك مع تيلدا. أنا Mariem, أتمنى أن يكون يومك سعيد. 11:10 PM, 31st Aug julie_ossama شكرا? 11:10 PM, 31st Aug Mariem هنوضح لحضرتك كل شيء 11:10 PM, 31st Aug Mariem حضرتك بتحتاجي تتوجهي للفرع لو مستخدم جديد لامضاء الاوراق من خلال الفرع 11:11 PM, 31st Aug julie_ossama فرع ايه؟ 11:11 PM, 31st Aug Mariem لو حضرتك مصرية، بتقدري تتوجهي لاي فرع مناسب ليكي 11:12 PM, 31st Aug Mariem نكون متواحدين فيه 11:12 PM, 31st Aug julie_ossama انا ظاهر عندى فى الapp on the run وbtech بس مش عارفة انا مفروض اروح اعمل اعمل ايه 11:12 PM, 31st Aug Mariem اي فرع منهم بس اهم شيء نكون متواجدين فيهم 11:12 PM, 31st Aug julie_ossama هما طاهرين ان دول اقربلى على الapp 11:13 PM, 31st Aug julie_ossama فكدا انتوا موجودين فيهم صح؟ 11:13 PM, 31st Aug julie_ossama او مكنوش هيظهرولى 11:13 PM, 31st Aug julie_ossama بروح اسأل على تليدا واقولهم انى عايزة امضى الورق ولا ايه؟ 11:13 PM, 31st Aug Mariem مظبوط لو ظاهرين بالفعل لحضرتك على التطبيق فا دا معناه اننا متواجدين بيهم 11:15 PM, 31st Aug Mariem بنكون متواجدين بداخل الفروع نفسها تقدري تسالي علينا اكيد 11:16 PM, 31st Aug julie_ossama اوكيه شكرا 11:16 PM, 31st Aug Mariem العفو تحت امرك 11:16 PM, 31st Aug Mariem اتمنى اننا نكون جاوبنا على كل أسئلتك! في أي استفسار اخر ممكن نساعدك فيها؟ 11:16 PM, 31st Aug julie_ossama لا شكرا? 11:16 PM, 31st Aug Mariem شكراً لتواصلك مع فريق دعم تيلدا! 11:18 PM, 31st Aug Take to Team Inbox",julie_ossama,Mariem Sherif,", I hope you are having a great day! أهلاً Julie_ossama, شكراً لتواصلك مع تيلدا. أنا Mariem, أتمنى أن يكون يومك سعيد. 11:10 PM, 31st Aug 31st Aug Mariem هنوضح لحضرتك كل شيء 11:10 PM, 31st Aug Mariem

In [12]:
print("\n=== ROWS WHERE NAME EXTRACTION FAILED ===")
empty_name_rows = df[df['Name'].isna()]
empty_name_rows



=== ROWS WHERE NAME EXTRACTION FAILED ===


,Ticket ID,Description,Name,Agent
10,1281384,مساء الخير انا كان معايا بطاقه تيلدا كانت مربو...,NaN,No Agent
68,1281442,NaN,NaN,No Agent
74,1281448,"Hi , Ticket ""#1280618 – I can't enter to my ac...",NaN,No Agent
113,1281487,م عارفه افتح الابلكيشن ولا اعمل الفيزا,NaN,No Agent
224,1281598,"Hi , I have a problem with my telda card that ...",NaN,No Agent
...,...,...,...,...
39805,1321192,السلام عليكم فريق دعم تيلدا، أنا عندي مشكلة في...,NaN,No Agent
39891,1321278,السلام عليكم، أنا حسابي على Telda اتقفل وبيظهر...,NaN,No Agent
39934,1321321,"i have input all the details of the card, yet...",NaN,No Agent
40093,1321480,ايه الفرق بين كروت تلدا,NaN,No Agent


In [16]:
df = pd.read_excel('telda_CS.xlsx')
df['Name'] = df['Subject'].str.extract(r'Conversation with (.+)')

In [18]:
df = df[df['Source'] == 'Chat']
df

,Ticket ID,Subject,Description,Status,Priority,Source,Type,Agent,Group,Created time,...,Sub Cat 2,Location,Telda Group Name,User ID,Mobile Number,@username,Telda Agent Name,App Version,OS,Name
0,1281374,Conversation with Anas Mohamed,Telda Bot Hi Anas ! Welcome to Telda. ? Please...,Resolved,Low,Chat,NaN,Abdelhameed Atef,Premium/Plus,2025-09-01 00:00:12,...,Profile limits,NaN,Premium/Plus,user_2bwjqSllrWvOjMq7RffXFaK1YPC,2.011135e+11,anasmoh7,abdelhameed.atef@telda.app,5.18,iOS,Anas Mohamed
1,1281375,Conversation with Mazen Babiker,Telda Bot Hi Mazen! Welcome to Telda. ? Please...,Resolved,Low,Chat,NaN,Radwa Talaat,Standard,2025-09-01 00:01:34,...,Foreigners,NaN,Standard,user_2v9eZYtiaub5rjdWriMHf1CkB8Y,2.011226e+11,mazenbabiker,radwa.talaat@telda.app,4.0.3,Android,Mazen Babiker
2,1281376,Conversation with Adhma Ahmed,"Adhma Ahmed العربية 10:54 PM, 31st Aug Telda B...",Resolved,Low,Chat,NaN,Mohamed Mostafa,Standard,2025-09-01 00:01:42,...,Peer to peer Out,NaN,Standard,user_31KU4ZVGEpjOy50LxN5nY6oSW4B,2.011251e+11,adhmaahmed545680,mohamed.mostafa@telda.app,5.20.5,iOS,Adhma Ahmed
3,1281377,Conversation with Marwan Amin,"Marwan Amin English 10:57 PM, 31st Aug Telda B...",Pending,Low,Chat,NaN,Sarah Abdelalim,Standard,2025-09-01 00:01:51,...,Eligibility,NaN,Standard,user_2waa0s6e8qDYeGNShWPXWjzPf7S,2.010278e+11,marwanamin056200,sarah.abdelalim@telda.app,5.15,iOS,Marwan Amin
4,1281378,Conversation with Yasmin Osama,Telda Bot Hi Yasmin! Welcome to Telda. ? Pleas...,Resolved,Low,Chat,NaN,Mohamed Mostafa,Premium/Plus,2025-09-01 00:03:16,...,How to Purchase Gift Cards,NaN,Premium/Plus,user_2tu1FAVfnwB3jTQUHvelpNBy4UY,2.012052e+11,yasminosam592906,mohamed.mostafa@telda.app,4.1.1,Android,Yasmin Osama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40164,1321551,Conversation with Omar Mohamed Rashad,Omar Mohamed Rashad انا دلوقتي عندي مشكله أن ب...,Pending,Low,Chat,NaN,Injy Walid,Standard,2025-09-30 23:54:28,...,OTP not received / delayed,NaN,Standard,user_2lUGG5cyYRoKUexlPN6bTn0xBbh,2.011532e+11,omarrrashad,injy.walid@telda.app,4.3.2,Android,Omar Mohamed Rashad
40165,1321552,Conversation with Yassin Walid,Yassin Walid * File attached - IMG_20250930_23...,Resolved,Low,Chat,NaN,Mariem Sherif,Standard,2025-09-30 23:54:31,...,Recover deleted account,NaN,Standard,user_2mWHGNZj1CqELCRws5qtprTLfW5,2.010118e+11,yassin1112,mariem.sherif@telda.app,4.4.0,Android,Yassin Walid
40166,1321553,Conversation with kero_.karam,"Mohamad مساء الخير 05:27 PM, 06th Nov Mohamad ...",Resolved,Low,Chat,NaN,Mariem Sherif,Social Media,2025-09-30 23:54:43,...,Delete account,NaN,Social Media,user id,0.000000e+00,username,mariem.sherif@telda.app,app version,platform,kero_.karam
40167,1321554,Conversation with Jasmine Ahmed,Jasmine Ahmed Hello where is the place where i...,Pending,Low,Chat,NaN,Sara Fadel,Premium/Plus,2025-09-30 23:54:58,...,IOS,NaN,Premium/Plus,user_2WRF7FXaCTsdSWUEpbkeilBwgNR,2.010085e+11,jasmine123,sara.fadel@telda.app,5.21.2,iOS,Jasmine Ahmed


In [19]:
first_data = df[['Ticket ID','Description', 'Name', 'Agent' ]]
empty_name_rows = first_data[first_data['Name'].isna()]
data_to_merge = df[['Ticket ID','Subject']]
merged_df = empty_name_rows.merge(data_to_merge, on='Ticket ID', how='left')
merged_df

,Ticket ID,Description,Name,Agent,Subject


In [14]:
import pandas as pd

# Example merge
merged_df = empty_name_rows.merge(data_to_merge, on='Ticket ID', how='left')
merged_df

,Ticket ID,Description,Name,Agent,Subject
0,1281384,مساء الخير انا كان معايا بطاقه تيلدا كانت مربو...,NaN,No Agent,تغيير رقم التليفون المربوط بيه الحساب
1,1281442,NaN,NaN,No Agent,لقد تم اغلاق البطاقه الخاصه بي ارجو تشغيل حساب...
2,1281448,"Hi , Ticket ""#1280618 – I can't enter to my ac...",NaN,No Agent,I can't enter to my account
3,1281487,م عارفه افتح الابلكيشن ولا اعمل الفيزا,NaN,No Agent,م عارفه افتح الابلكيشن ولا اعمل الفيزا
4,1281598,"Hi , I have a problem with my telda card that ...",NaN,No Agent,NaN
...,...,...,...,...,...
868,1321192,السلام عليكم فريق دعم تيلدا، أنا عندي مشكلة في...,NaN,No Agent,NaN
869,1321278,السلام عليكم، أنا حسابي على Telda اتقفل وبيظهر...,NaN,No Agent,NaN
870,1321321,"i have input all the details of the card, yet...",NaN,No Agent,hello i cannot deposit money from a friends card?
871,1321480,ايه الفرق بين كروت تلدا,NaN,No Agent,ايه الفرق بين كروت تلدا


In [7]:
print("=== CHECKING FOR EMPTY CELLS IN NAME COLUMN ===")
print(f"Total rows: {len(df)}")
print(f"Rows with NaN in Name column: {df['Name'].isna().sum()}")
print(f"Rows with valid names: {df['Name'].notna().sum()}")
print("\n=== SAMPLE OF ROWS WITH EMPTY NAMES ===")
empty_name_rows = df[df['Name'].isna()]
print(f"Found {len(empty_name_rows)} rows with empty names")
if len(empty_name_rows) > 0:
    print("\nFirst 5 rows with empty names:")
    print(empty_name_rows[['Description', 'Name', 'Agent']].head())
    
print("\n=== SAMPLE OF ROWS WITH VALID NAMES ===")
valid_name_rows = df[df['Name'].notna()]
if len(valid_name_rows) > 0:
    print("First 5 rows with valid names:")
    print(valid_name_rows[['Description', 'Name', 'Agent']].head())

=== CHECKING FOR EMPTY CELLS IN NAME COLUMN ===
Total rows: 40169
Rows with NaN in Name column: 873
Rows with valid names: 39296

=== SAMPLE OF ROWS WITH EMPTY NAMES ===
Found 873 rows with empty names

First 5 rows with empty names:
                                           Description Name     Agent
10   مساء الخير انا كان معايا بطاقه تيلدا كانت مربو...  NaN  No Agent
68                                                 NaN  NaN  No Agent
74   Hi , Ticket "#1280618 – I can't enter to my ac...  NaN  No Agent
113             م عارفه افتح الابلكيشن ولا اعمل الفيزا  NaN  No Agent
224  Hi , I have a problem with my telda card that ...  NaN  No Agent

=== SAMPLE OF ROWS WITH VALID NAMES ===
First 5 rows with valid names:
                                         Description           Name  \
0  Telda Bot Hi Anas ! Welcome to Telda. ? Please...   Anas Mohamed   
1  Telda Bot Hi Mazen! Welcome to Telda. ? Please...  Mazen Babiker   
2  Adhma Ahmed العربية 10:54 PM, 31st Aug Telda B...    Adhma

In [16]:
df = pd.read_excel('telda_CS.xlsx')df['Name'] = df['Subject'].str.extract(r'Conversation with (.+)')
df

,Ticket ID,Subject,Description,Status,Priority,Source,Type,Agent,Group,Created time,...,Summary,Sub Cat 2,Location,Telda Group Name,User ID,Mobile Number,@username,Telda Agent Name,App Version,OS
0,1281374,Conversation with Anas Mohamed,"Telda Bot Hi Anas ! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:52 PM, 31st Aug Anas Mohamed العربية 10:52 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:52 PM, 31st Aug Anas Mohamed تحدث لمندوبنا 10:53 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:53 PM, 31st Aug Anas Mohamed انا معدي الليمت مفرود كمان ٨ دقايق تفتح ؟؟ 10:53 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:53 PM, 31st Aug Anas Mohamed تمم 10:54 PM, 31st Aug Telda Hello Anas , Thanks for reaching out to Telda! This is Abdelhameed, I hope you are having a great day! أهلاً Anas , شكراً لتواصلك مع تيلدا. أنا Abdelhameed, أتمنى أن يكون يومك سعيد. 10:54 PM, 31st Aug Anas Mohamed تمم انا معدي ليمت الشهر مفرود هتتفتح كمان ٥ دقايق صخ 10:54 PM, 31st Aug Anas Mohamed ف بدايه الشهر 10:54 PM, 31st Aug Abdelhameed متقلقش هساعدك في استفسارك وهوضحلك التفاصيل 10:55 PM, 31st Aug Abdelhameed استاذنك الانتظار من 3 دقائق إلى 5 دقائق أراجع الموضوع 10:55 PM, 31st Aug Anas Mohamed تمام 10:55 PM, 31st Aug Abdelhameed ? 10:55 PM, 31st Aug Anas Mohamed بسرعه بس 10:55 PM, 31st Aug Abdelhameed براجع الموضوع وهرجعلك مره تانيه متقلقش 10:56 PM, 31st Aug Anas Mohamed قالولي يوم 9-1 مفرود كمان ٢ دقيقتين صح كده 10:56 PM, 31st Aug Anas Mohamed ؟ 10:58 PM, 31st Aug Abdelhameed شكرا لانتظارك وبعتذرلك عن التاخير 10:58 PM, 31st Aug Anas Mohamed ولا يهمك 10:58 PM, 31st Aug Abdelhameed بعد الساعة 12 هيتم تحديث الحد الاقصى للحساب 10:58 PM, 31st Aug Anas Mohamed تمم 10:58 PM, 31st Aug Abdelhameed اقدر اساعدك في اي استفسار تاني ؟ 10:58 PM, 31st Aug Abdelhameed !مجرد تنبيه بسيط: المحادثة بتقفل لو مفيش رد خلال دقيقتين، بس إحنا دايمًا موجودين لو احتجتنا 10:59 PM, 31st Aug Abdelhameed يبدو أن وقت المحادثة انتهى، هنقفل الشات دلوقتى. اطمن لو احتجت أي حاجة، كلمنا في أي وقت – إحنا دايمًا موجودين علشان نساعدك 11:00 PM, 31st Aug Take to Team Inbox",Resolved,Low,Chat,NaN,Abdelhameed Atef,Premium/Plus,2025-09-01 00:00:12,...,No Summary,Profile limits,NaN,Premium/Plus,user_2bwjqSllrWvOjMq7RffXFaK1YPC,2.011135e+11,anasmoh7,abdelhameed.atef@telda.app,5.18,iOS
1,1281375,Conversation with Mazen Babiker,"Telda Bot Hi Mazen! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:54 PM, 31st Aug Mazen Babiker العربية 10:54 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker تحدث لمندوبنا 10:54 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker عاوز اعرف انا اجنبي وعندي اقامه وبسبورد وللعلم عمري 16 هل بعملها بس بالاتنين دول؟ 10:55 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:55 PM, 31st Aug Mazen Babiker تمام 10:55 PM, 31st Aug Telda Hello Mazen , Thanks for reaching out to Telda! This is Radwa, I hope you are having a great day! أهلاً Mazen , شكراً لتواصلك مع تيلدا. أنا Radwa, أتمنى أن يكون يومك سعيد. 10:55 PM, 31st Aug Mazen Babiker هل اقدر استلم البطاقه بس بالاقامه والبسبورد للعلم عمري 16 10:56 PM, 31st Aug Mazen Babiker انا مش مصري اجنبي 10:56 PM, 31st Aug Radwa حابيين نطمنك هيتم الرد علي جميع استفساراتك و مساعدتك في توضيح الامر 10:57 PM, 31st Aug Mazen Babiker تمام 10:57 PM, 31st Aug Radwa متاح استلام كارت تيلدا من سن 15 سنه بجواز سفر واقامه ساريين 10:58 PM, 31st Aug Radwa المستند الأساسي المطلوب هو جواز سفر ساري و تقديم إقامة سارية مع بعض المستندات الأخرى بنائاً على الوظيفه. وفي بعض الحالات الفردية، قد يُطلب منك تقديم مستندات إضافية في الفرع، مثل إثبات الدخل ، وقد يحتاج البنك إلى مستندات أخرى إضافية. 10:58 PM, 31st Aug Mazen Babiker تمام 10:58 PM, 31st Aug Radwa اتمنى اننا نكون جاوبنا على كل أسئلتك! في أي استفسار اخر ممكن نساعدك فيها؟ 10:59 PM, 31st Aug Radwa !مجرد تنبيه بسيط: المحادثة بت

In [11]:

# Extract full name after 'Conversation with '
df['Name'] = df['Subject'].str.extract(r'Conversation with (.+)')

df


,Ticket ID,Subject,Description,Status,Priority,Source,Type,Agent,Group,Created time,...,Sub Cat 2,Location,Telda Group Name,User ID,Mobile Number,@username,Telda Agent Name,App Version,OS,Name
0,1281374,Conversation with Anas Mohamed,Telda Bot Hi Anas ! Welcome to Telda. ? Please...,Resolved,Low,Chat,NaN,Abdelhameed Atef,Premium/Plus,2025-09-01 00:00:12,...,Profile limits,NaN,Premium/Plus,user_2bwjqSllrWvOjMq7RffXFaK1YPC,2.011135e+11,anasmoh7,abdelhameed.atef@telda.app,5.18,iOS,Anas Mohamed
1,1281375,Conversation with Mazen Babiker,Telda Bot Hi Mazen! Welcome to Telda. ? Please...,Resolved,Low,Chat,NaN,Radwa Talaat,Standard,2025-09-01 00:01:34,...,Foreigners,NaN,Standard,user_2v9eZYtiaub5rjdWriMHf1CkB8Y,2.011226e+11,mazenbabiker,radwa.talaat@telda.app,4.0.3,Android,Mazen Babiker
2,1281376,Conversation with Adhma Ahmed,"Adhma Ahmed العربية 10:54 PM, 31st Aug Telda B...",Resolved,Low,Chat,NaN,Mohamed Mostafa,Standard,2025-09-01 00:01:42,...,Peer to peer Out,NaN,Standard,user_31KU4ZVGEpjOy50LxN5nY6oSW4B,2.011251e+11,adhmaahmed545680,mohamed.mostafa@telda.app,5.20.5,iOS,Adhma Ahmed
3,1281377,Conversation with Marwan Amin,"Marwan Amin English 10:57 PM, 31st Aug Telda B...",Pending,Low,Chat,NaN,Sarah Abdelalim,Standard,2025-09-01 00:01:51,...,Eligibility,NaN,Standard,user_2waa0s6e8qDYeGNShWPXWjzPf7S,2.010278e+11,marwanamin056200,sarah.abdelalim@telda.app,5.15,iOS,Marwan Amin
4,1281378,Conversation with Yasmin Osama,Telda Bot Hi Yasmin! Welcome to Telda. ? Pleas...,Resolved,Low,Chat,NaN,Mohamed Mostafa,Premium/Plus,2025-09-01 00:03:16,...,How to Purchase Gift Cards,NaN,Premium/Plus,user_2tu1FAVfnwB3jTQUHvelpNBy4UY,2.012052e+11,yasminosam592906,mohamed.mostafa@telda.app,4.1.1,Android,Yasmin Osama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40164,1321551,Conversation with Omar Mohamed Rashad,Omar Mohamed Rashad انا دلوقتي عندي مشكله أن ب...,Pending,Low,Chat,NaN,Injy Walid,Standard,2025-09-30 23:54:28,...,OTP not received / delayed,NaN,Standard,user_2lUGG5cyYRoKUexlPN6bTn0xBbh,2.011532e+11,omarrrashad,injy.walid@telda.app,4.3.2,Android,Omar Mohamed Rashad
40165,1321552,Conversation with Yassin Walid,Yassin Walid * File attached - IMG_20250930_23...,Resolved,Low,Chat,NaN,Mariem Sherif,Standard,2025-09-30 23:54:31,...,Recover deleted account,NaN,Standard,user_2mWHGNZj1CqELCRws5qtprTLfW5,2.010118e+11,yassin1112,mariem.sherif@telda.app,4.4.0,Android,Yassin Walid
40166,1321553,Conversation with kero_.karam,"Mohamad مساء الخير 05:27 PM, 06th Nov Mohamad ...",Resolved,Low,Chat,NaN,Mariem Sherif,Social Media,2025-09-30 23:54:43,...,Delete account,NaN,Social Media,user id,0.000000e+00,username,mariem.sherif@telda.app,app version,platform,kero_.karam
40167,1321554,Conversation with Jasmine Ahmed,Jasmine Ahmed Hello where is the place where i...,Pending,Low,Chat,NaN,Sara Fadel,Premium/Plus,2025-09-30 23:54:58,...,IOS,NaN,Premium/Plus,user_2WRF7FXaCTsdSWUEpbkeilBwgNR,2.010085e+11,jasmine123,sara.fadel@telda.app,5.21.2,iOS,Jasmine Ahmed


In [21]:
print(df.columns.tolist())

['Ticket ID', 'Subject', 'Description', 'Status', 'Priority', 'Source', 'Type', 'Agent', 'Group', 'Created time', 'Due by Time', 'Resolved time', 'Closed time', 'Last update time', 'Initial response time', 'Time tracked', 'First response time (in hrs)', 'Resolution time (in hrs)', 'Agent interactions', 'Customer interactions', 'Resolution status', 'First response status', 'Tags', 'Survey results', 'Product', 'Main Category', 'Every response status', 'Product.1', 'Summary', 'Sub Cat 2', 'Location', 'Telda Group Name', 'User ID', 'Mobile Number', '@username', 'Telda Agent Name', 'App Version', 'OS', 'Name']


In [12]:
df = df.head(5)
df = df[['Description', 'Name', 'Agent' ]]
df

,Description,Name,Agent
0,Telda Bot Hi Anas ! Welcome to Telda. ? Please...,Anas Mohamed,Abdelhameed Atef
1,Telda Bot Hi Mazen! Welcome to Telda. ? Please...,Mazen Babiker,Radwa Talaat
2,"Adhma Ahmed العربية 10:54 PM, 31st Aug Telda B...",Adhma Ahmed,Mohamed Mostafa
3,"Marwan Amin English 10:57 PM, 31st Aug Telda B...",Marwan Amin,Sarah Abdelalim
4,Telda Bot Hi Yasmin! Welcome to Telda. ? Pleas...,Yasmin Osama,Mohamed Mostafa


In [22]:
pd.set_option('display.max_colwidth', None)

df.head()

,Ticket ID,Description,Name,Agent,Cleaned_Description
0,1281374,"Telda Bot Hi Anas ! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:52 PM, 31st Aug Anas Mohamed العربية 10:52 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:52 PM, 31st Aug Anas Mohamed تحدث لمندوبنا 10:53 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:53 PM, 31st Aug Anas Mohamed انا معدي الليمت مفرود كمان ٨ دقايق تفتح ؟؟ 10:53 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:53 PM, 31st Aug Anas Mohamed تمم 10:54 PM, 31st Aug Telda Hello Anas , Thanks for reaching out to Telda! This is Abdelhameed, I hope you are having a great day! أهلاً Anas , شكراً لتواصلك مع تيلدا. أنا Abdelhameed, أتمنى أن يكون يومك سعيد. 10:54 PM, 31st Aug Anas Mohamed تمم انا معدي ليمت الشهر مفرود هتتفتح كمان ٥ دقايق صخ 10:54 PM, 31st Aug Anas Mohamed ف بدايه الشهر 10:54 PM, 31st Aug Abdelhameed متقلقش هساعدك في استفسارك وهوضحلك التفاصيل 10:55 PM, 31st Aug Abdelhameed استاذنك الانتظار من 3 دقائق إلى 5 دقائق أراجع الموضوع 10:55 PM, 31st Aug Anas Mohamed تمام 10:55 PM, 31st Aug Abdelhameed ? 10:55 PM, 31st Aug Anas Mohamed بسرعه بس 10:55 PM, 31st Aug Abdelhameed براجع الموضوع وهرجعلك مره تانيه متقلقش 10:56 PM, 31st Aug Anas Mohamed قالولي يوم 9-1 مفرود كمان ٢ دقيقتين صح كده 10:56 PM, 31st Aug Anas Mohamed ؟ 10:58 PM, 31st Aug Abdelhameed شكرا لانتظارك وبعتذرلك عن التاخير 10:58 PM, 31st Aug Anas Mohamed ولا يهمك 10:58 PM, 31st Aug Abdelhameed بعد الساعة 12 هيتم تحديث الحد الاقصى للحساب 10:58 PM, 31st Aug Anas Mohamed تمم 10:58 PM, 31st Aug Abdelhameed اقدر اساعدك في اي استفسار تاني ؟ 10:58 PM, 31st Aug Abdelhameed !مجرد تنبيه بسيط: المحادثة بتقفل لو مفيش رد خلال دقيقتين، بس إحنا دايمًا موجودين لو احتجتنا 10:59 PM, 31st Aug Abdelhameed يبدو أن وقت المحادثة انتهى، هنقفل الشات دلوقتى. اطمن لو احتجت أي حاجة، كلمنا في أي وقت – إحنا دايمًا موجودين علشان نساعدك 11:00 PM, 31st Aug Take to Team Inbox",Anas Mohamed,Abdelhameed Atef,", I hope you are having a great day! أهلاً Anas , شكراً لتواصلك مع تيلدا. أنا Abdelhameed, أتمنى أن يكون يومك سعيد. 10:54 PM, 31st Aug 31st Aug 31st Aug Abdelhameed متقلقش هساعدك في استفسارك وهوضحلك التفاصيل 10:55 PM, 31st Aug Abdelhameed استاذنك الانتظار من 3 دقائق إلى 5 دقائق أراجع الموضوع 10:55 PM, 31st Aug 31st Aug Abdelhameed ? 10:55 PM, 31st Aug 31st Aug Abdelhameed براجع الموضوع وهرجعلك مره تانيه متقلقش 10:56 PM, 31st Aug 31st Aug 31st Aug Abdelhameed شكرا لانتظارك وبعتذرلك عن التاخير 10:58 PM, 31st Aug 31st Aug Abdelhameed بعد الساعة 12 هيتم تحديث الحد الاقصى للحساب 10:58 PM, 31st Aug 31st Aug Abdelhameed اقدر اساعدك في اي استفسار تاني ؟ 10:58 PM, 31st Aug Abdelhameed !مجرد تنبيه بسيط: المحادثة بتقفل لو مفيش رد خلال دقيقتين، بس إحنا دايمًا موجودين لو احتجتنا 10:59 PM, 31st Aug Abdelhameed يبدو أن وقت المحادثة انتهى، هنقفل الشات دلوقتى. اطمن لو احتجت أي حاجة، كلمنا في أي وقت – إحنا دايمًا موجودين علشان نساعدك 11:00 PM, 31st Aug Take to Team Inbox"
1,1281375,"Telda Bot Hi Mazen! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:54 PM, 31st Aug Mazen Babiker العربية 10:54 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker تحدث لمندوبنا 10:54 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker عاوز اعرف انا اجنبي وعندي اقامه وبسبورد وللعلم عمري 16 هل بعملها بس بالاتنين دول؟ 10:55 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:55 PM, 31st Aug Mazen Babiker تمام 10:55 PM, 31st Aug Telda Hello Mazen , Thanks for reaching out to Telda! This is Radwa, I hope you are having a great day! أهلاً Mazen , شكراً لتواصلك مع تيلدا. أنا Radwa, أتمنى أن يكون يومك سعيد. 10:55 PM, 31st Aug Mazen Babiker هل اقدر استلم البطاقه بس بالاقامه والبسبورد للعلم عمري 16 10:56 PM, 31st Aug Mazen Babiker انا مش مصري اجنبي 10:56 PM, 31st Aug Radwa حابيين نطمنك هيتم الرد علي جميع استفساراتك و مساعدتك في

In [9]:
# Check for empty cells in Name column and show original Subject
print("=== CHECKING FOR EMPTY CELLS IN NAME COLUMN ===")
print(f"Total rows: {len(df)}")
print(f"Rows with NaN in Name column: {df['Name'].isna().sum()}")
print(f"Rows with valid names: {df['Name'].notna().sum()}")

print("\n=== SUBJECT VALUES WHERE NAME EXTRACTION FAILED ===")
empty_name_rows = df[df['Name'].isna()]
print(f"Found {len(empty_name_rows)} rows where Name extraction failed")
if len(empty_name_rows) > 0:
    print("\nSubject values that didn't match 'Conversation with (.+)' pattern:")
    print("-" * 80)
    for idx, subject in empty_name_rows['Subject'].items():
        print(f"Row {idx}: '{subject}'")

=== CHECKING FOR EMPTY CELLS IN NAME COLUMN ===
Total rows: 40169
Rows with NaN in Name column: 873
Rows with valid names: 39296

=== SUBJECT VALUES WHERE NAME EXTRACTION FAILED ===
Found 873 rows where Name extraction failed

Subject values that didn't match 'Conversation with (.+)' pattern:
--------------------------------------------------------------------------------


KeyError: 'Subject'

In [13]:
import pandas as pd
import re

# Example DataFrame

def clean_description(row):
    name = re.escape(row['Name'])
    text = row['Description']

    # Step 1: remove client segments
    cleaned = re.sub(fr'{name}[^,]*,', '', text)

    # Step 2: handle "this is {first_name}"
    if pd.notna(row['Agent']):
        first_name = row['Agent'].split()[0]  # extract first name
        pattern = fr'.*?(this\s+is\s+{re.escape(first_name)}\b)'
        match = re.search(pattern, cleaned, flags=re.IGNORECASE)
        if match:
            # Remove everything before "this is first_name"
            cleaned = cleaned[match.end():].strip()
    
    return cleaned

# Apply
df['Cleaned_Description'] = df.apply(clean_description, axis=1)




In [17]:
pd.set_option('display.max_colwidth', None)
df

,Ticket ID,Subject,Description,Status,Priority,Source,Type,Agent,Group,Created time,...,Summary,Sub Cat 2,Location,Telda Group Name,User ID,Mobile Number,@username,Telda Agent Name,App Version,OS
0,1281374,Conversation with Anas Mohamed,"Telda Bot Hi Anas ! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:52 PM, 31st Aug Anas Mohamed العربية 10:52 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:52 PM, 31st Aug Anas Mohamed تحدث لمندوبنا 10:53 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:53 PM, 31st Aug Anas Mohamed انا معدي الليمت مفرود كمان ٨ دقايق تفتح ؟؟ 10:53 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:53 PM, 31st Aug Anas Mohamed تمم 10:54 PM, 31st Aug Telda Hello Anas , Thanks for reaching out to Telda! This is Abdelhameed, I hope you are having a great day! أهلاً Anas , شكراً لتواصلك مع تيلدا. أنا Abdelhameed, أتمنى أن يكون يومك سعيد. 10:54 PM, 31st Aug Anas Mohamed تمم انا معدي ليمت الشهر مفرود هتتفتح كمان ٥ دقايق صخ 10:54 PM, 31st Aug Anas Mohamed ف بدايه الشهر 10:54 PM, 31st Aug Abdelhameed متقلقش هساعدك في استفسارك وهوضحلك التفاصيل 10:55 PM, 31st Aug Abdelhameed استاذنك الانتظار من 3 دقائق إلى 5 دقائق أراجع الموضوع 10:55 PM, 31st Aug Anas Mohamed تمام 10:55 PM, 31st Aug Abdelhameed ? 10:55 PM, 31st Aug Anas Mohamed بسرعه بس 10:55 PM, 31st Aug Abdelhameed براجع الموضوع وهرجعلك مره تانيه متقلقش 10:56 PM, 31st Aug Anas Mohamed قالولي يوم 9-1 مفرود كمان ٢ دقيقتين صح كده 10:56 PM, 31st Aug Anas Mohamed ؟ 10:58 PM, 31st Aug Abdelhameed شكرا لانتظارك وبعتذرلك عن التاخير 10:58 PM, 31st Aug Anas Mohamed ولا يهمك 10:58 PM, 31st Aug Abdelhameed بعد الساعة 12 هيتم تحديث الحد الاقصى للحساب 10:58 PM, 31st Aug Anas Mohamed تمم 10:58 PM, 31st Aug Abdelhameed اقدر اساعدك في اي استفسار تاني ؟ 10:58 PM, 31st Aug Abdelhameed !مجرد تنبيه بسيط: المحادثة بتقفل لو مفيش رد خلال دقيقتين، بس إحنا دايمًا موجودين لو احتجتنا 10:59 PM, 31st Aug Abdelhameed يبدو أن وقت المحادثة انتهى، هنقفل الشات دلوقتى. اطمن لو احتجت أي حاجة، كلمنا في أي وقت – إحنا دايمًا موجودين علشان نساعدك 11:00 PM, 31st Aug Take to Team Inbox",Resolved,Low,Chat,NaN,Abdelhameed Atef,Premium/Plus,2025-09-01 00:00:12,...,No Summary,Profile limits,NaN,Premium/Plus,user_2bwjqSllrWvOjMq7RffXFaK1YPC,2.011135e+11,anasmoh7,abdelhameed.atef@telda.app,5.18,iOS
1,1281375,Conversation with Mazen Babiker,"Telda Bot Hi Mazen! Welcome to Telda. ? Please select your preferable language? الرجاء اختيار اللغة المفضلة لك؟ 10:54 PM, 31st Aug Mazen Babiker العربية 10:54 PM, 31st Aug Telda Bot كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker تحدث لمندوبنا 10:54 PM, 31st Aug Telda Bot ممكن توضح الأستفسار الخاص بك أو كيف يمكننا مساعدتك ؟ 10:54 PM, 31st Aug Mazen Babiker عاوز اعرف انا اجنبي وعندي اقامه وبسبورد وللعلم عمري 16 هل بعملها بس بالاتنين دول؟ 10:55 PM, 31st Aug Telda Bot شكراً لتوضيح أستفسارك, سيتم تحويلك لممثل خدمة العملاء خلال لحظات. ?? 10:55 PM, 31st Aug Mazen Babiker تمام 10:55 PM, 31st Aug Telda Hello Mazen , Thanks for reaching out to Telda! This is Radwa, I hope you are having a great day! أهلاً Mazen , شكراً لتواصلك مع تيلدا. أنا Radwa, أتمنى أن يكون يومك سعيد. 10:55 PM, 31st Aug Mazen Babiker هل اقدر استلم البطاقه بس بالاقامه والبسبورد للعلم عمري 16 10:56 PM, 31st Aug Mazen Babiker انا مش مصري اجنبي 10:56 PM, 31st Aug Radwa حابيين نطمنك هيتم الرد علي جميع استفساراتك و مساعدتك في توضيح الامر 10:57 PM, 31st Aug Mazen Babiker تمام 10:57 PM, 31st Aug Radwa متاح استلام كارت تيلدا من سن 15 سنه بجواز سفر واقامه ساريين 10:58 PM, 31st Aug Radwa المستند الأساسي المطلوب هو جواز سفر ساري و تقديم إقامة سارية مع بعض المستندات الأخرى بنائاً على الوظيفه. وفي بعض الحالات الفردية، قد يُطلب منك تقديم مستندات إضافية في الفرع، مثل إثبات الدخل ، وقد يحتاج البنك إلى مستندات أخرى إضافية. 10:58 PM, 31st Aug Mazen Babiker تمام 10:58 PM, 31st Aug Radwa اتمنى اننا نكون جاوبنا على كل أسئلتك! في أي استفسار اخر ممكن نساعدك فيها؟ 10:59 PM, 31st Aug Radwa !مجرد تنبيه بسيط: المحادثة بت